# Pythonによる画像解析ノート Vol. 5

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import imageio
from PIL import Image as pilimage
from IPython.display import display, Image

# Jupyter notebookの中で画像を表示させるための関数、読み込んだ画像が16-bitならば画像を8bitに変換するように修正
def show(img_grey):
    if img_grey.dtype == "uint16":
        img_grey = cv2.normalize(img_grey, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    _, buf = cv2.imencode(".jpg", img_grey)
    print(type(buf))
    print(buf.shape)
    display(Image(data = buf.tobytes()))

## ファイル操作  
Pythonの標準ライブラリの1つであるosモジュールをインポートすると、様々なファイル操作ができる。

In [2]:
import os
print(os.path.exists("test.txt")) # existsはファイルが存在するか確認
print(os.path.isfile("bank.csv")) # isfileはファイルかどうかを確かめる
print(os.path.isdir("images")) # isdirはディレクトリ（フォルダ）かどうかを確かめる
print(os.getcwd()) # getcwd()でcurrent working directoryを取得
# os.rename("bank.csv", "renamed_bank.csv")  # ファイル名を変更したい場合はrenameを使う

False
False
True
/Users/masak_takaine/Dropbox/MT_Python


### フォルダの作成  
`os.mkdir()`は既存のフォルダを上書きできないので、存在を確認してから作成。フォルダの削除には`os.rmdir()`を使う。

In [3]:
from os import path
cwd = os.getcwd()
test_dir = os.path.join(cwd, "test_dir") # os.path.joinを使うと勝手にseparatorを挟む
if not os.path.exists(test_dir):  # dirSGが存在しなければ作成する
    os.mkdir(test_dir)         
        
os.rmdir(test_dir) # フォルダの削除

### ファイル名リストを取得  
`os.listdir()`関数でフォルダ内のファイルのリストを取得。フォルダ名(folder_name)は文字列型にする必要がある。  
サブフォルダがある場合はサブフォルダ内のファイル名もリストアップされる。  
リスト内包表記を使用すると、特定の拡張子のファイル名だけを抽出することができる。  
split(".")でピリオドの前後２つの単語に分かれる。[-1]は後ろの単語（=拡張子）を表す。[1]にしても同じ

In [4]:
folder_name = "images"
filelist = os.listdir(str(folder_name)) # str型にする必要あり

# tifファイルだけを抽出
tif_files = [f for f in filelist if f.split(".")[-1] == "tif"]
print(tif_files)

['mri-stack.tif', 'foci.tif', 'organ-of-corti.tif', 'dots.tif', 'Cell_Colony.tif', '3ch_image.tif', 'yeast_tl.tif', 'AuPbSn40.tif', 'yeast_2ch_regi.tif', 'Dot_Blot.tif', 'MAX_mitosis.tif', 'embryos.tif', 'cell_16bit.tif', 'yeast_2ch.tif', '2ch_image.tif', 'twitter_icon-1.tif', '2ch11z.tif', 'MAX_mitosis-1.tif', 'FluorescentCells.tif', 'ramp_rgb.tif', 'dots_large.tif', 'rgb_z15.tif']


ライブラリglobのglob()関数でも、フォルダ中のファイル名のリストを取得できる。  
glob()は正規表現にマッチするパス名を返すので、特定の拡張子のファイルのみを取得することも可能。サブフォルダのファイル名もリストアップ。

In [5]:
import glob
path = os.path.join(cwd, "python_automation")
file_list = glob.glob(path + "/" + "*") #フォルダ内の全てのfile/folderのリストを取得
excelfile_list = glob.glob(path + "/" + "*.xlsx") #フォルダ内のdocxファイルのリストを取得

print("ファイルの数:", len(file_list))
print("Excelファイル:", excelfile_list)

ファイルの数: 7
Excelファイル: ['/Users/masak_takaine/Dropbox/MT_Python/python_automation/9x9.xlsx', '/Users/masak_takaine/Dropbox/MT_Python/python_automation/renzoku.xlsx', '/Users/masak_takaine/Dropbox/MT_Python/python_automation/hello.xlsx', '/Users/masak_takaine/Dropbox/MT_Python/python_automation/cell_w.xlsx', '/Users/masak_takaine/Dropbox/MT_Python/python_automation/test100.xlsx']


`os.path.basename()`関数を使うと上位のフォルダ名を除いたファイル名が取得できる

In [6]:
basenames = [os.path.basename(file) for file in excelfile_list] # リスト内包表記
basenames

['9x9.xlsx', 'renzoku.xlsx', 'hello.xlsx', 'cell_w.xlsx', 'test100.xlsx']

### ファイル名から拡張子を除く  
方法1: 正規表現を使用して、ピリオドとそれに続く任意の3文字を消去。

In [7]:
import re    # 正規表現モジュールreをインポート
re.sub("\\....", "", "test.jpg")    

'test'

方法2: 関数os.path.splittext()を利用。ピリオドで分割して1番目の要素を抽出。

In [8]:
from os import path 
os.path.splitext("test.jpg")[0]

'test'

方法3: 関数split()を利用。ピリオドで分割して1番目の要素を抽出

In [9]:
filename = "test.jpg" 
filename.split(".")[0]  # ピリオドで分割して、画像の名前を取得

'test'

方法4: スライス表記を利用。先頭から、最後から5番目の文字までを抽出（最後の4文字を消去）。

In [10]:
"test.jpg"[:-4]

'test'

## GUIによるパラメータの入力  
ImageJ/FIJIではScript parameter記法が使用できたが、Pythonでは使用不可。  
代わりに`PySimpleGUI`ライブラリを使用してGUIのウインドウを立ち上げて、さまざまなパラメータを指定できるようにする。  
https://www.pysimplegui.org/en/latest/cookbook/  

PysimpleGUIでは`one-shot window`と`persistent window`がある、パラメータ1つだけを入力するなら前者の方が便利。  

In [11]:
# One-shot windowの例
import PySimpleGUI as sg

layout = [
   [sg.Text("My one-shot window.")],
   [sg.InputText(key="-IN-")],
   [sg.Submit(), sg.Cancel()],
]

window = sg.Window("Window Title", layout)

event, values = window.read()
window.close()

text_input = values["-IN-"]
sg.popup("You entered", text_input)

'OK'

In [12]:
# Persistent windowの例
import PySimpleGUI as sg

sg.theme("BluePurple")

layout = [
   [sg.Text("Your typed chars appear here:"), sg.Text(size=(15, 1), key="-OUTPUT-")],
   [sg.Input(key="-IN-")],
   [sg.Button("Show"), sg.Button("Exit")],
]

window = sg.Window("Pattern 2B", layout)

while True:  # Event Loop
   event, values = window.read()
   print(event, values)
   if event in (None, "Exit"):
       break
   if event == "Show":
       # "-OUTPUT-" 要素を "-IN-" 要素の値に更新
       window["-OUTPUT-"].update(values["-IN-"])

window.close()

Exit {'-IN-': ''}


In [21]:
# Date, Source folder, Destination folderの情報をvaluesに辞書形式で格納
import PySimpleGUI as sg
layout = [[sg.Text("Date of experiments:"), sg.InputText(key="date")],
   [sg.Text("Source folder:"), sg.InputText(key="folderS")],[sg.FolderBrowse(target="folderS")],
    [sg.Text("Destination folder:"), sg.InputText(key="folderD")],[sg.FolderBrowse(target="folderD")],
          [sg.Submit(), sg.Cancel()],
]
window = sg.Window("Choose folders", layout)
event, values = window.read()
window.close()
print(values)

{'date': '230810', 'folderS': '', 'Browse': '', 'folderD': '', 'Browse0': ''}


## バッチ処理  
実際の画像解析においては、大量の画像データを扱う。  
1例として、あるフォルダ内の特定の拡張子を持つ画像ファイル全てにフィルタ処理をして、  
得られた画像を別のフォルダに保存するというバッチ処理を考える。

In [14]:
import os
from os import path
import glob
from skimage.filters import gaussian
from skimage import img_as_ubyte

In [15]:
cwd = os.getcwd()
path = os.path.join(cwd, "images")
file_list = glob.glob(path + "/" + "*") #フォルダ内の全てのfile/folderのリストを取得

方法1: ファイルを1つずつ読みこんで、フィルタ処理した後に保存

In [16]:
# 画像を保存するフォルダを作成
target_dir = os.path.join(cwd, "blurred") # os.path.joinを使うと勝手にseparatorを挟む
if not os.path.exists(target_dir):  # target_dirが存在しなければ作成する
    os.mkdir(target_dir)     

In [17]:
for file in file_list:
    if ".jpg" in file or  ".JPG" in file:
        basename = os.path.basename(file) # フォルダ名無しのファイル名
        filename = basename.split(".")[0] # さらに拡張子無しのファイル名
        # print(filename)
        img = cv2.imread(file)
        smoothed_image = img_as_ubyte(gaussian(img, sigma=3, mode='constant', cval=0.0, channel_axis=2))
        savepath = os.path.join(cwd, target_dir,"smoothed_"+filename+".png")
        cv2.imwrite(savepath, smoothed_image)

方法2: フィルタ処理画像をndarrayに変換してリストに格納してから保存  
こちらの方が機械学習でよく使われる

In [18]:
# 画像を保存するフォルダを作成
target_dir = os.path.join(cwd, "blurred") # os.path.joinを使うと勝手にseparatorを挟む
if not os.path.exists(target_dir):  # dirSGが存在しなければ作成する
    os.mkdir(target_dir)   

In [19]:
img_list = []
name_list = []
for file in file_list:
    if ".jpg" in file or ".JPG" in file:
        basename = os.path.basename(file)
        filename = basename.split(".")[0]
        # print(filename)
        img = cv2.imread(file)
        smoothed_image = img_as_ubyte(gaussian(img, sigma=3, mode='constant', cval=0.0, channel_axis=2))
        img_list.append(smoothed_image)
        name_list.append(filename)

for img, name in zip(img_list, name_list):
    savepath = os.path.join(cwd, target_dir,"smoothed_"+name+".png")
    cv2.imwrite(savepath, img)

#### バッチ処理において、読み込むファイルの入ったフォルダ（source folder）と保存先のフォルダ（destination folder）をGUIで指定できるようにする

In [20]:
import PySimpleGUI as sg
import numpy as np
import scipy as sp
import cv2
import glob
import os
from os import path
from skimage.filters import gaussian
from skimage import img_as_ubyte

layout = [
   [sg.Text("Source folder:"), sg.InputText(key="folderS")],[sg.FolderBrowse(target="folderS", initial_folder= os.getcwd())],
    [sg.Text("Destination folder:"), sg.InputText(key="folderD")],[sg.FolderBrowse(target="folderD", initial_folder= os.getcwd())],
          [sg.Submit(), sg.Cancel()],
]
window = sg.Window("Choose folders", layout)
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Cancel'):
        break
    # elif event == 'Submit':
    else:
        path = values["folderS"]
        target_dir = values["folderD"]
        file_list = glob.glob(path + "/" + "*") #フォルダ内の全てのfile/folderのリストを取得
        img_list = []
        name_list = []
        for file in file_list:
            if ".jpg" in file or ".JPG" in file:
                basename = os.path.basename(file)
                filename = basename.split(".")[0]
                # print(filename)
                img = cv2.imread(file)
                smoothed_image = img_as_ubyte(gaussian(img, sigma=3, mode='constant', cval=0.0, channel_axis=2))
                img_list.append(smoothed_image)
                name_list.append(filename)

        for img, name in zip(img_list, name_list):
            savepath = os.path.join(target_dir,"smoothed_"+name+".png")
            cv2.imwrite(savepath, img)
        print("Done.")
        break
        
window.close()